In [1]:
import yfinance as yf
import pandas_ta as ta
import nsepy as nse
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from datetime import date
import vectorbt as vbt
import backtrader as bt
from dateutil.relativedelta import relativedelta

In [41]:
nifty = "NIFTY"
nifty_fut = nse.get_history(symbol=nifty,
                        start=date(2022,4,21),
                        end=date(2022,4,28),
                        index=True,
                        option_type='CE',
                        strike_price=17400,
                        expiry_date=date(2022,4,28))

nifty_fut

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2022-04-21,NIFTY,2022-04-28,CE,17400.0,73.7,173.95,73.7,159.25,165.0,159.25,372037,3.262557e+11,2.583526e+09,2454300,113200,17392.60
2022-04-22,NIFTY,2022-04-28,CE,17400.0,99.0,119.00,48.8,53.45,52.0,53.45,1379660,1.205570e+12,5.266094e+09,5094400,2640100,17171.95


In [79]:
today = date.today()
option_data = pd.read_csv('nifty_15_yr_data.csv',  parse_dates=True, index_col='Date')
option_data.sort_index(inplace=True)
option_data.shape

(155846, 16)

In [76]:
# define and month year range to loop over
month_list = np.arange((option_data.index[-1].month+1)%12, 13, step = 1); print(month_list)
yr_list = np.arange(option_data.index[-1].year + 1, today.year+1, step = 1 ); print(yr_list)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2021 2022]


In [81]:
# break the loop into 2 parts to avoid having to rerun everything if there are querying errors
for yr in yr_list:
    # loop through all the months and years
    counter = 0
    print('Year: ', yr)
    for mnth in month_list:
        current_dt = date(yr, mnth, 1)
        start_dt = current_dt + relativedelta(months = -2)
        end_dt = max(nse.get_expiry_date(year = yr, month = mnth))
        
#         print('current: ', current_dt)
#         print('start: ', start_dt)
#         print('end: ', end_dt)
        
        # get nifty futures data
        nifty_fut = nse.get_history(symbol = nifty,
                               start = start_dt, end = end_dt,
                               index = True,
                               expiry_date = end_dt)
        nifty_data = nifty_data.append(nifty_fut)
        
        # calculate high and low values for each month; round off to get strike prices
        high = nifty_fut['Close'].max()
        high = int(round(high/100)*100) + 1000# ; print('High:', high)
        
        low = nifty_fut['Close'].min()
        low = int(round(low/100)*100) + 1000# ; print('Low :', low)
        
        for strike in range(low, high, 100): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            print(f'Months: {counter}, strike: {strike}')
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            nifty_opt = nse.get_history(symbol = nifty,
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
            time.sleep(random.randint(1,10)) # pause for random interval so as to not overwhelm the site
            nifty_opt = nse.get_history(symbol = nifty,
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
#             print(option_data)
            
        counter+=1
       
        # print(month)   

Year:  2021
Months: 0, strike: 12700
Months: 0, strike: 12800
Months: 0, strike: 12900
Months: 0, strike: 13000
Months: 0, strike: 13100
Months: 0, strike: 13200
Months: 0, strike: 13300
Months: 0, strike: 13400
Months: 0, strike: 13500
Months: 0, strike: 13600
Months: 0, strike: 13700
Months: 0, strike: 13800
Months: 0, strike: 13900
Months: 0, strike: 14000
Months: 0, strike: 14100
Months: 0, strike: 14200
Months: 0, strike: 14300
Months: 0, strike: 14400
Months: 0, strike: 14500
Months: 0, strike: 14600
Months: 0, strike: 14700
Months: 0, strike: 14800
Months: 0, strike: 14900
Months: 0, strike: 15000
Months: 0, strike: 15100
Months: 0, strike: 15200
Months: 0, strike: 15300
Months: 0, strike: 15400
Months: 0, strike: 15500
Months: 1, strike: 14100
Months: 1, strike: 14200
Months: 1, strike: 14300
Months: 1, strike: 14400
Months: 1, strike: 14500
Months: 1, strike: 14600
Months: 1, strike: 14700
Months: 1, strike: 14800
Months: 1, strike: 14900
Months: 1, strike: 15000
Months: 1, st

ExpiryDateError: No expiry date found in the month of 2022-2 for vix derivatives

In [90]:
nse.get_expiry_date(year = 2022, month = 2)

ReadTimeout: HTTPSConnectionPool(host='www1.nseindia.com', port=443): Read timed out. (read timeout=None)

In [ ]:
option_data.to_csv("nifty_15_yr_options.csv")